In [ ]:
import sys
sys.path.append("../")  # add parent directory to the system path
from utils_data import *
from utils_model import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from time import time

root_name = "Research"


# ________________ GET THE DATA READY ________________ #
train_path, test_path, val_path = make_data_path(root_name=root_name)
DataGenerator = ImageDataGenerator(rescale=1.0 / 255)
train_data, test_data, val_data = make_data_ready(
    DataGenerator, train_path, test_path, val_path
)

# ________________ CONFIG THE BASE MODELS ________________ #
weights = "imagenet"
input_shape = (224, 224, 3)
num_class = 524

include_top = False
base_trainable = False
pooling = "max"
learning_rate = 1
epochs = 1

model_funcs = [
    tf.keras.applications.mobilenet.MobileNet,
]

model_0s = {}

for model_func in model_funcs:
    model_name = model_func.__name__

    model_0s[model_name] = configure_model_base(
        model_func=model_func,
        weights=weights,
        include_top=include_top,
        base_trainable=base_trainable,
        input_shape=input_shape,
        pooling=pooling,
        num_class=num_class,
        learning_rate=learning_rate,
    )

MobileNet = model_0s["MobileNet"]

# MobileNet.summary()

# freeze all layers in the nested MobileNet model
for layer in MobileNet.layers[1].layers:
    layer.trainable = False

# recompile the outer model to apply changes
MobileNet.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate * .1),
    metrics=["accuracy"],
)

# test 0
MobileNet_history0 = MobileNet.fit(train_data,
                                  epochs=epochs,
                                  steps_per_epoch=len(train_data),
                                  validation_data=val_data,
                                  validation_steps=int(0.25 * len(val_data))
                                  )

test0_train_accuracy = MobileNet_history0.history["accuracy"]
test0_val_accuracy = MobileNet_history0.history["val_accuracy"]

test0_train_loss = MobileNet_history0.history["loss"]
test0_val_loss = MobileNet_history0.history["val_loss"]

test0_test_loss, test0_test_accuracy = MobileNet.evaluate(test_data)

# recompile the outer model to apply changes
MobileNet.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate * .1),
    metrics=["accuracy"],
)

# test 1
callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience = 5, restore_best_weights = True),
    
    tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.7, patience = 2, verbose = 1),
]


MobileNet_history1 = MobileNet.fit(train_data,
                                  epochs=epochs,
                                  steps_per_epoch=len(train_data),
                                  validation_data=val_data,
                                  validation_steps=int(0.25 * len(val_data)),
                                  verbose = 1, 
                                  callbacks=callbacks
                                  )

test1_train_accuracy = MobileNet_history1.history["accuracy"]
test1_val_accuracy = MobileNet_history1.history["val_accuracy"]

test1_train_loss = MobileNet_history1.history["loss"]
test1_val_loss = MobileNet_history1.history["val_loss"]

test1_test_loss, test1_test_accuracy = MobileNet.evaluate(test_data)

metrics = {
    "test0" : {
        "train_accuracy" : test0_train_accuracy,
        "val_accuracy" : test0_val_accuracy,        
        "train_loss" : test0_train_loss,
        "val_loss" : test0_val_loss,  
        "test_accuracy" : test0_test_accuracy,
        "test_loss" : test0_test_loss,      
        "execution_time" : 0,
    },
    
    "test1" : {
        "train_accuracy" : test1_train_accuracy,
        "val_accuracy" : test1_val_accuracy,        
        "train_loss" : test1_train_loss,
        "val_loss" : test1_val_loss,   
        "test_accuracy" : test1_test_accuracy,
        "test_loss" : test1_test_loss,      
        "execution_time" : 0,
    }    
}

# ________________ SAVE MODEL PERFORMANCE LOG TO .JSON ________________ #
test_name = "mobilenet_test"
file_path = "../Research/output/data"
file_name = "model_performance_log_" + test_name + ".json"
save_metric(metrics, file_name, file_path)

# ________________ SAVE MODEL PERFORMANCE PLOT TO .jpg ________________ #
nrows = 2
ncols = 2
width = 10
height = nrows * width / ncols  # to shape each plot element in square shape

figure = visualize_metric(metrics, nrows=nrows, ncols=ncols, figsize=(width, height))

file_path = "../Research/output/viz"
file_name = "model_performance_plot_" + test_name + ".jpg"

In [ ]:
# Now, print the trainability status to verify
for layer_number, layer in enumerate(MobileNet.layers[1].layers):
    print(layer_number, layer.name, layer.trainable)
